In [1]:
# Import all the required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# Loading the MNIST dataset
train_dataset = datasets.MNIST(root='./', train = True, 
                               transform=transforms.ToTensor(),
                               download = True)
test_dataset = datasets.MNIST(root='./', train = False, 
                               transform=transforms.ToTensor(),
                               download = True)

In [3]:
#Setting the batch Size
batch_size = 128

# Initializing the data loader

train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

In [4]:
model = nn.Sequential(
          nn.Linear(28*28, 520),
          nn.ReLU(),
          nn.Linear(520, 320),
          nn.ReLU(),
          nn.Linear(320, 240),
          nn.ReLU(),
          nn.Linear(240, 120),
          nn.ReLU(),
          nn.Linear(120, 10),
        )

In [5]:
# Defining the loss function
loss_fn = nn.CrossEntropyLoss()

In [6]:
# Defining the optimization algorithm

optimizer = optim.RMSprop(model.parameters(), lr=1e-4)
correct = 0

from __future__ import print_function

In [7]:
# Start training the network now
model.train()

for epoch in range(0, 10):
    correct = 0
    for batch_idx, (data, target) in enumerate(train_data_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data.view(-1, 784))
        loss = loss_fn(output, target)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
    print('Train Epoch: {} \tLoss: {:.6f}\tAccuracy: {}/{} ({:.2f}%)'.format(
                epoch, loss.item(), correct, len(train_data_loader.dataset),
                100. * float(correct) / len(train_data_loader.dataset)))
           


Train Epoch: 0 	Loss: 0.499040	Accuracy: 50373/60000 (83.95%)
Train Epoch: 1 	Loss: 0.261324	Accuracy: 54704/60000 (91.17%)
Train Epoch: 2 	Loss: 0.232330	Accuracy: 55960/60000 (93.27%)
Train Epoch: 3 	Loss: 0.141290	Accuracy: 56735/60000 (94.56%)
Train Epoch: 4 	Loss: 0.051175	Accuracy: 57366/60000 (95.61%)
Train Epoch: 5 	Loss: 0.065793	Accuracy: 57768/60000 (96.28%)
Train Epoch: 6 	Loss: 0.100536	Accuracy: 58031/60000 (96.72%)
Train Epoch: 7 	Loss: 0.132401	Accuracy: 58285/60000 (97.14%)
Train Epoch: 8 	Loss: 0.047600	Accuracy: 58541/60000 (97.57%)
Train Epoch: 9 	Loss: 0.083770	Accuracy: 58730/60000 (97.88%)


In [8]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_data_loader:
        data, target = Variable(data), Variable(target)
        output = model(data.view(-1, 784))
        # sum up batch loss
        test_loss += loss_fn(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()    


In [9]:
test_loss /= len(test_data_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
    test_loss, correct, len(test_data_loader.dataset),
    100. * float(correct) / len(test_data_loader.dataset)))


Test set: Average loss: 0.0007, Accuracy: 9709/10000 (97.09%)

